В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.


In [1]:
from sklearn.datasets import load_digits, load_breast_cancer
digits = load_digits()

In [9]:
split = int(digits.data.shape[0] * .75)

X = digits.data
y = digits.target

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

### Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeigborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [14]:
def evclid(a, b):
    return sum((a - b) ** 2)


In [20]:
res = {}

for x in X_test:
    res[tuple(x)] = []
    for index, xx in enumerate(X_train):
        res[tuple(x)].append((evclid(x, xx), y_train[index]))
    res[tuple(x)].sort()


In [25]:
def sosed(a):
    return res[tuple(a)][0][1]


In [34]:
err = 0
for i, x in enumerate(X_test):
    err += sosed(x) != y_test[i]

print err
print len(X_test)
print float(err) / len(X_test)

with open('task1.txt', 'w') as answ:
    answ.write('0.0377777777778')

17
450
0.0377777777778


### Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [37]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)

In [39]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
err = 0
for i, x in enumerate(clf.predict(X_test)):
    err += x != y_test[i]

print err
print len(X_test)
print float(err) / len(X_test)

with open('task2.txt', 'w') as answ:
    answ.write('0.0644444444444')

29
450
0.0644444444444
